
# TensorFlow Object Detection: Подготовка данных для обучения

https://github.com/tensorflow/models/tree/master/research/object_detection

### Монтирование Google Drive
Для данного демо нам понадобятся несколько ноутбуков, которые должны работать с одними и теме же данными. Поэтому, чтобы все ноутбуки имели доступ к нужным данным, нам будет необходимо подключить диск Google Drive и сохранять все данные на нём (включая данные, скачанные из интернета).

Для монтирования диска нужно выполнить данный блок, перейти по ссылке, получить код, скопировать его в поле ниже (в этом блоке) и нажать Enter

После монтирования диск будет находиться здесь: `/content/drive/My Drive`

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Рабочая директория
Все данные будем хранить в директории `/content/drive/My Drive/tf_od_demo` (TensorFlow Object Detection Demo)

При первом запуске создадим директорию (если её еще не существует), в противном случае надо заменить True на False.

При последующих подключениях к диску (в том числе в других ноутбуках) директорию создавать не надо, в ней уже будут сохранены все данные, которые мы туда поместили.

In [2]:
if True:
    !mkdir "/content/drive/My Drive/tf_od_demo"
%cd "/content/drive/My Drive/tf_od_demo"

mkdir: cannot create directory ‘/content/drive/My Drive/tf_od_demo’: File exists
/content/drive/My Drive/tf_od_demo


### Подготовка библиотеки `object_detection`
Библиотека `object_detection` находится в репозитории `tensorflow/models` в разделе `research`

Необходимо склонировать код библиотеки и сконфигурировать модели (сбилдить прото модели).

Этот шаг нужно сделать один раз (не повторять, если папка `models` уже находится в текущей директории).

Подробнее: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md

In [3]:
pip install tf_slim

     |████████████████████████████████| 352 kB 7.7 MB/s 


In [4]:
pip install lvis

In [5]:
pip install tf-models-official

     |████████████████████████████████| 1.8 MB 8.4 MB/s 
     |████████████████████████████████| 37.1 MB 47 kB/s 
     |████████████████████████████████| 679 kB 72.3 MB/s 
     |████████████████████████████████| 4.3 MB 55.3 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 636 kB 62.9 MB/s 
     |████████████████████████████████| 1.2 MB 62.9 MB/s 
     |████████████████████████████████| 99 kB 13.9 MB/s 
     |████████████████████████████████| 211 kB 69.2 MB/s 
     |████████████████████████████████| 90 kB 13.5 MB/s 
     |████████████████████████████████| 454.4 MB 10 kB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 454.3 MB 18 kB/s 
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dm-tree to determine which version is compatible with o

In [ ]:
if True:
  
    !git clone https://github.com/tensorflow/models
    !cd models/research && protoc object_detection/protos/*.proto --python_out=.
    !cd models/research && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim && python object_detection/builders/model_builder_test.py

Cloning into 'models'...
remote: Enumerating objects: 60242, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 60242 (delta 1), reused 7 (delta 0), pack-reused 60234
Receiving objects: 100% (60242/60242), 573.74 MiB | 13.85 MiB/s, done.
Resolving deltas: 100% (41927/41927), done.
Checking out files: 100% (2596/2596), done.
2021-08-17 05:27:31.880065: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


### Загрузка библиотек
Загрузка TensorFlow и других библиотек. Кроме того, загрузка модуля `dataset_util` из пакета `object_detection`, который будет нужен для создания датасета в нужном формате.

In [6]:
import pandas as pd
import os
from PIL import Image

import tensorflow as tf

import sys
sys.path.insert(0, 'models/research')

from object_detection.utils import dataset_util

### Функция для создания одного обучающего образца
В этой функции создаётся экземпляр класса `tf.train.Example`, который соответствует одной обучающей картике. Ей могут соответствовать несколько ground-truth баундинг боксов. Однако, конкретно в данном примере на картинке есть строго один бокс. В противном случае списки `xmins`, `xmaxs`, `ymins`, `ymaxs`, `classes_text`, `classes` должны иметь соответствующее количество элементов ( = кол-ву боксов на данной картинке).

Создавать экземпляры класса `tf.train.Example` можно произвольным способом. В данном примере на вход в функцию подаётся строка из CSV файла (`annot.csv`). Главное -- заполнить соовтестсвующие поля словаре `feature={...}`

Подробнее: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md

In [7]:
def create_tf_example(example):
  
    img_fpath = os.path.join('my_data', example.id)
    img = Image.open(img_fpath)
    height = img.size[1]
    width = img.size[0]
    filename = str.encode(example.id)
    with open(img_fpath, mode='rb') as f:
        encoded_image_data = f.read()
    image_format = b'jpeg'

    # List of normalized left x coordinates in bounding box (1 per box)
    xmins = [example.xmin1 / float(width), example.xmin2 / float(width), example.xmin3 / float(width),] 
    # List of normalized right x coordinates in bounding box # (1 per box)
    xmaxs = [example.xmax1 / float(width), example.xmax2 / float(width), example.xmax3 / float(width)] 
    # List of normalized top y coordinates in bounding box (1 per box)
    ymins = [example.ymin1 / float(height), example.ymin2 / float(height), example.ymin3 / float(height)] 
    # List of normalized bottom y coordinates in bounding box # (1 per box)
    ymaxs = [example.ymax1 / float(height), example.ymax2 / float(height), example.ymax3 / float(height)] 
    # List of string class name of bounding box (1 per box)
    classes_text = [b'Flash', b'Marker', b'Souvenir']
    # List of integer class id of bounding box (1 per box)
    classes = [1,2,3]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

### Чтение CSV файла с разметкой
В данном файле представлена разметка обучающих изображений. Сам файл и его формат показаны лишь для примера, они никак не связаны с библиотекой `object_detection`. Наша финальная цель -- создать датасет в формате `TFRecord`, состоящий из экземпляров `tf.train.Example`.

---

В данном примере формат файла annot.csv следующий (один бокс на файл):

id,xmin,ymin,xmax,ymax

1.jpg,261,260,601,615

2.jpg,130,429,401,734

...

---

Перед запуском этого блока загрузите необходимые данные (папка `my_data`) в текущую рабочую директорию (tf_od_demo). Один из вариантов, как это можно сделать, это загрузить архив `my_data.7z`, а затем разархивировать его с помощью команды:

`!7z x my_data.7z`

In [ ]:
if True:
    !7z x 10.7_my_data.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         
ERROR: No more files
10.7_my_data.7z



System ERROR:
Unknown error -2147024872


In [8]:
annot = pd.read_csv('./my_data/annot.csv', sep=';')
annot.head()

,id,xmin1,ymin1,xmax1,ymax1,xmin2,ymin2,xmax2,ymax2,xmin3,ymin3,xmax3,ymax3
0,1.jpg,390,580,560,720,740,350,925,470,950,255,1070,350
1,2.jpg,180,450,390,500,650,370,710,670,920,450,1040,520
2,3.jpg,190,710,440,770,830,230,930,320,890,370,1080,430
3,4.jpg,1000,330,1100,430,790,200,960,330,220,650,480,840
4,5.jpg,260,660,500,720,900,310,1020,430,440,250,580,340


### Создание TFRecord
Здесь мы создаём финальный датасет в формате `TFRecord`, который необходим для запуска обучения TF Object Detection. 

В цикле по всем обучающим образцам создаем `TF Example` и записываем его в `TF Record`.

In [9]:
writer = tf.io.TFRecordWriter('my_data/train_data.record')

for idx, row in annot.iterrows():
    tf_example = create_tf_example(row)
    writer.write(tf_example.SerializeToString())

writer.close()